In [1]:
import os
import os.path as op
import json
import cv2
import numpy as np
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageFont

In [2]:
root_dir = '/media/stopmosk/data/huawei'

img_root = op.join(root_dir, 'datasets_orig/huawei_1000/images')
gen_root = op.join(root_dir, 'output_dl2')

gen_filename = 'pred.huawei_1000.test.beam5.max20.odlabels.eval.json'  #_coco_format.json'

gen_fullpaths = []
gen_subroot = op.join('coco_ru_base_xe_3', 'checkpoint-28-250000')

#gen_fullpaths.append(op.join(gen_root, 'stock1', gen_filename))
#gen_fullpaths.append(op.join(gen_root, 'agg_1cc_10coco_base_xe1', 'checkpoint-3-350000', gen_filename))
#gen_fullpaths.append(op.join(gen_root, 'cc_base_xe_2', 'checkpoint-15-200000', gen_filename))
#gen_fullpaths.append(op.join(gen_root, 'stage_2_results', gen_filename))
#gen_fullpaths.append(op.join(gen_root, 'a100_agg_1', 'checkpoint-24-23000', gen_filename))
#gen_fullpaths.append(op.join(gen_root, 'coco140', gen_filename))
gen_fullpaths.append(op.join(gen_root, gen_subroot, gen_filename))
gen_fullpaths[-1]

'/media/stopmosk/data/huawei/output_dl2/coco_ru_base_xe_3/checkpoint-28-250000/pred.huawei_1000.test.beam5.max20.odlabels.eval.json'

In [3]:
res_dicts = []

for gen_fullpath in gen_fullpaths:
    with open(gen_fullpath) as fp:
        res_json = json.load(fp)

    res_dict = {s['image_id']: s['caption'] for s in res_json}
    res_dicts.append(res_dict)
    #key = list(res_dict.keys())[0]
    #res_dict[key]
    # res_dict


In [4]:
def show_res(sample_idx, show=True, save=False):
    im_filename = sample_idx
    im_fullpath = os.path.join(img_root, im_filename)
    im = Image.open(im_fullpath)
    im.thumbnail((600, 400))

    plt.figure(figsize=(16,10))
    draw = ImageDraw.Draw(im)
    unicode_font = ImageFont.truetype("DejaVuSansMono.ttf", 12)
    font_yshift = -2

    y_shift = 0
    colors = [(255,0,0,255), (0,255,0,255), (0,0,255,255), (0,0,128,255), (255,0,255,255), (0,0,0,255)]
    for pred_num, res_dict in enumerate(res_dicts):
        text_cap = f"{res_dict[sample_idx]}"
        #print('\n' + text_cap)

        max_str_len = im.width // 7  #6  # max string length in letters
        strings_num = len(text_cap) // max_str_len + 1

        b_text = [0, y_shift , im.width, y_shift + 12 * strings_num]
        try:
            draw.rectangle(b_text, fill='red' if pred_num==0 else 'blue')
        except:
            print(pred_num, colors[pred_num])
            exit()

        text_cap_chunks = [text_cap[seek: seek + max_str_len] for seek in range(0, len(text_cap), max_str_len)]
        for i, chunk in enumerate(text_cap_chunks):
            draw.text((0, font_yshift + y_shift + i * 12), chunk, font=unicode_font, stroke_width=1, stroke_fill=255)
        y_shift += (i + 1) * 12
        
    plt.axis('off')
    
    if show:
        plt.imshow(im);
    
    if save:
        plt.savefig(op.join(export_dir, sample_idx), bbox_inches='tight')
        plt.close()


In [5]:
%matplotlib inline

export_dir = op.join(gen_root, gen_subroot, 'draw_results') 
os.makedirs(export_dir, exist_ok=True)

for sample_idx in tqdm(list(res_dict.keys())[0:]):
    show_res(sample_idx, save=True)

 10%|█         | 103/998 [00:16<03:03,  4.89it/s]

0 (255, 0, 0, 255)


 12%|█▏        | 115/998 [00:18<02:25,  6.08it/s]

0 (255, 0, 0, 255)
0 (255, 0, 0, 255)


 59%|█████▊    | 585/998 [01:27<00:53,  7.66it/s]

0 (255, 0, 0, 255)


 68%|██████▊   | 680/998 [01:41<00:56,  5.62it/s]

0 (255, 0, 0, 255)


100%|██████████| 998/998 [02:27<00:00,  6.75it/s]
